
#All I've done so far is prep the OG dataset for gathering census data.
---



In [ ]:
pip install pgeocode

In [ ]:
import pandas as pd
import pgeocode
import requests

In [ ]:
#defining all functions for use with census API.
#retrieving city and state for each zip code.
def getCityStateByZip(zipCode):
    location = nomi.query_postal_code(zipCode)
    if location is not None and pd.notna(location.place_name) and pd.notna(location.state_name):
        return location.place_name.lower().strip(), location.state_name.upper().strip()
    else:
        return None, None
#converting state names into their abbreviations.
def convertStateToAbbrv(stateName):
    states = {"ALABAMA": "AL", "ALASKA": "AK", "ARIZONA": "AZ", "ARKANSAS": "AR", "CALIFORNIA": "CA", "COLORADO": "CO", "CONNECTICUT": "CT", "DELAWARE": "DE", "FLORIDA": "FL",
              "GEORGIA": "GA", "HAWAII": "HI", "IDAHO": "ID", "ILLINOIS": "IL", "INDIANA": "IN", "IOWA": "IA", "KANSAS": "KS", "KENTUCKY": "KY", "LOUISIANA": "LA", "MAINE": "ME",
              "MARYLAND": "MD", "MASSACHUSETTS": "MA", "MICHIGAN": "MI", "MINNESOTA": "MN", "MISSISSIPPI": "MS", "MISSOURI": "MO", "MONTANA": "MT", "NEBRASKA": "NE", "NEVADA": "NV",
              "NEW HAMPSHIRE": "NH", "NEW JERSEY": "NJ", "NEW MEXICO": "NM", "NEW YORK": "NY", "NORTH CAROLINA": "NC", "NORTH DAKOTA": "ND", "OHIO": "OH", "OKLAHOMA": "OK", "OREGON": "OR",
              "PENNSYLVANIA": "PA", "RHODE ISLAND": "RI", "SOUTH CAROLINA": "SC", "SOUTH DAKOTA": "SD", "TENNESSEE": "TN", "TEXAS": "TX", "UTAH": "UT", "VERMONT": "VT", "VIRGINIA": "VA",
              "WASHINGTON": "WA", "WEST VIRGINIA": "WV", "WISCONSIN": "WI", "WYOMING": "WY"}
    return states.get(stateName.upper(), None)
#getting FIPS codes using the census gazetteer files, defining descriptors for location names, splitting geoid column after first two numbers.
def getFIPSCodes(city, state):
    if not pd.isna(city) and not pd.isna(state):
        city = city.strip().lower()
        state = state.strip().upper()
        descriptors = ["", " city", " town", " cdp", " township", "borough", "village"]
        for desc in descriptors:
            match = gazetteer[(gazetteer["NAME"].str.contains(rf"^{city}{desc}$", na=False, regex=True)) & (gazetteer["USPS"] == state)]
            if not match.empty:
                geoID = match.iloc[0]["GEOID"]
                stateFIPS = geoID[:2]
                placeFIPS = geoID[2:]
                return stateFIPS, placeFIPS
    return None, None
#querying the census API to collect the variables defined below.
def queryCensusAPI(stateFIPS, placeFIPS):
    params = {"get": ",".join(variables), "for": f"place:{placeFIPS}", "in": f"state:{stateFIPS}", "key": apiKey}
    response = requests.get(baseURL, params=params)

In [ ]:
#loading the texas roadhouse and census gazetteer datasets, extracting the zip code from the state column,
#and initializing pgeocode for zip codes.
trDf = pd.read_csv("texasRoadhouseLocations.csv")
gazetteer = pd.read_csv("2020censusPlaces.txt", delimiter="\t", dtype=str)
trDf["ZIP"] = trDf["state"].str.split().str[1]
nomi = pgeocode.Nominatim("us")
#populating city and state using zip codes and converting state names to abbrevations.
trDf["city"] = trDf["ZIP"].apply(lambda z: getCityStateByZip(z)[0])
trDf["state"] = trDf["ZIP"].apply(lambda z: getCityStateByZip(z)[1])
trDf["state"] = trDf["state"].apply(convertStateToAbbrv)
#normalizing gazetteer data, adding FIPS columns.
gazetteer["NAME"] = gazetteer["NAME"].str.strip().str.lower()
gazetteer["USPS"] = gazetteer["USPS"].str.strip().str.upper()
trDf["stateFIPS"] = None
trDf["placeFIPS"] = None

In [ ]:
#populating FIPS codes for city and state and saving the output to trFIPS.
for index, row in trDf.iterrows():
    city = row["city"]
    state = row["state"]
    if state in gazetteer["USPS"].values:
        stateFIPS, placeFIPS = getFIPSCodes(city, state)
    else:
        stateFIPS, placeFIPS = None, None
    trDf.at[index, "stateFIPS"] = stateFIPS
    trDf.at[index, "placeFIPS"] = placeFIPS

trDf.to_csv("trFIPS.csv", index=False)

In [ ]:
#defining api credentials, URL, data to be obtained (variables), and descriptive column names (columnMapping).
apiKey = "apiKey goes here"
baseURL = "https://api.census.gov/data/2020/acs/acs5"
variables = ["NAME","B01001_001E", "B01001_002E", "B01001_026E", "B01002_001E", "B01002_002E", "B01002_003E", "B02001_002E", "B02001_003E",
             "B02001_005E", "B03001_003E", "B19013_001E", "B19001_002E", "B19001_003E", "B19001_004E", "B19301_001E", "B17001_002E", "B25064_001E",
             "B25002_001E", "B25003_002E", "B25003_003E", "B25077_001E", "B25031_001E", "B25035_001E", "B15003_001E", "B15003_017E", "B15003_022E",
             "B15003_023E", "B23025_003E", "C24010_001E", "B24011_004E", "B24011_034E", "B03002_012E", "B03002_004E", "B03002_006E", "B08301_001E",
             "B08013_001E", "B08201_001E", "B18101_001E", "B27010_001E", "B11001_001E", "B11003_001E", "B11009_001E", "B12001_001E"]
columnMapping = {"NAME": "placeName", "B01001_001E": "totalPopulation", "B01001_002E": "malePopulation", "B01001_026E": "femalePopulation", "B01002_001E": "medianAge",
                 "B01002_002E": "medianAgeMale", "B01002_003E": "medianAgeFemale", "B02001_002E": "whitePopulation", "B02001_003E": "blackPopulation", "B02001_005E": "asianPopulation",
                 "B03001_003E": "hispanicPopulation", "B19013_001E": "medianHouseholdIncome", "B19001_002E": "incomeBelow10k", "B19001_003E": "income10kTo15k",
                 "B19001_004E": "income15kTo25k", "B19301_001E": "perCapitaIncome", "B17001_002E": "populationBelowPoverty", "B25064_001E": "medianGrossRent",
                 "B25002_001E": "totalHousingUnits", "B25003_002E": "ownerOccupiedHousingUnits", "B25003_003E": "renterOccupiedHousingUnits", "B25077_001E": "medianHomeValue",
                 "B25031_001E": "medianMonthlyHousingCosts", "B25035_001E": "medianYearStructureBuilt", "B15003_001E": "totalEducationPopulation", "B15003_017E": "bachelorsDegree",
                 "B15003_022E": "mastersDegree", "B15003_023E": "doctorateDegree", "B23025_003E": "employedPopulation", "C24010_001E": "occupationData",
                 "B24011_004E": "managementOccupation", "B24011_034E": "serviceOccupation", "B03002_012E": "foreignBornPopulation", "B03002_004E": "notHispanicWhitePopulation",
                 "B03002_006E": "notHispanicBlackPopulation", "B08301_001E": "commuters", "B08013_001E": "meanTravelTime", "B08201_001E": "carAvailability",
                 "B18101_001E": "disabilityPopulation", "B27010_001E": "healthInsurancePopulation", "B11001_001E": "householdPopulation", "B11003_001E": "singleParentHouseholds",
                 "B11009_001E": "unmarriedPartners", "B12001_001E": "marriedPopulation"}
results = []

In [ ]:
#loading the trFIPS dataset created above and requesting data from the API.
trhDf = pd.read_csv("trFIPS.csv")
for _, row in trhDf.iterrows():
    try:
        if pd.isna(row["stateFIPS"]) or pd.isna(row["placeFIPS"]):
            print(f"Skipping row with missing FIPS: {row}")
            continue
        stateFIPS = str(int(row["stateFIPS"])).zfill(2)
        placeFIPS = str(int(row["placeFIPS"])).zfill(5)
        data = queryCensusAPI(stateFIPS, placeFIPS)
        header = data[0]
        for item in data[1:]:
            rowData = {"stateFIPS": stateFIPS, "placeFIPS": placeFIPS}
            rowData.update({header[i]: item[i] for i in range(len(header))})
            results.append(rowData)
    except Exception as e:
        print(f"Error processing row: {row}, Error: {e}")
#saving the data to a dataframe, renaming the columns as defined above and saving the results to censusData.csv.
censusData = pd.DataFrame(results)
censusData.rename(columns=columnMapping, inplace=True)
censusData.to_csv("censusData.csv", index=False)

In [ ]:
#loading the census dataset.
censusDf = pd.read_csv("censusData.csv")  # Demographic data

In [ ]:
#ensuring stateFIPS and placeFIPS are integers in both texas roadhouse and census data.
trhDf["stateFIPS"] = trhDf["stateFIPS"].astype("Int64")
trhDf["placeFIPS"] = trhDf["placeFIPS"].astype("Int64")
censusDf["stateFIPS"] = censusDf["stateFIPS"].astype("Int64")
censusDf["placeFIPS"] = censusDf["placeFIPS"].astype("Int64")
#performing an inner join to merge the datasets on stateFIPS and placeFIPS.
df = pd.merge(trhDf, censusDf, on=["stateFIPS", "placeFIPS"], how="inner")
#dropping duplicate rows and saving the cleaned dataset as trhCensusDf.csv.
df = df.drop_duplicates()
print(f"Number of rows after dropping duplicates: {len(df)}")
df.to_csv("trhCensusDf.csv", index=False)